In [ ]:
import boto3, sagemaker
import pandas as pd

In [ ]:
smrt = boto3.client('sagemaker-runtime')
sess = sagemaker.Session()

In [ ]:
df = pd.read_csv('mnist/test.csv', chunksize=1000)

In [ ]:
results_sub = []
counter = 0
for chunk in df:
    payload = chunk.to_csv(header=False, index=False)
    response = smrt.invoke_endpoint(
        EndpointName="mnist-endopoint-1",
        Body=payload.encode('utf8'),
        ContentType='text/csv'
    )
    
    results = response['Body'].read().decode('utf-8')
    results_final = list(map(lambda x: x, results.split('\n')))
    results_sub += [y for y in results_final if y]
    
    counter += chunk.shape[0]
    print(f'Predicted {counter}')

In [ ]:
results_final_submition = list(zip(range(1, len(results_sub) + 1), results_sub))
submitions = pd.DataFrame(results_final_submition)
submitions.columns = ['ImageId', 'Label']
submitions.to_csv('kaggle_submition.csv', index=False)

In [ ]:
# sess.delete_endpoint(endpoint_name="mnist-endopoint-1")